### Databases selected  

For this project, I decided to analyze Airbnb data from Seattle. 
In this notebook, the three datasets of the Seattle Airbnb Open Data are analyzed and processed.  
Two new .csv files with the final version of the Seattle dataset (one **WITH**, the other **WITH NOT** encoded categorical features) are the final output of this notebook. The two datasets will be used as input in the other notebooks.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

### Import Seattle datasets

In [2]:
seattle_calendar = pd.read_csv('Seattle/calendar.csv')
seattle_listings = pd.read_csv('Seattle/listings.csv')
seattle_reviews = pd.read_csv('Seattle/reviews.csv')

### Explore the datasets:

In [3]:
print(f'The Seattle calendar dataset has {seattle_calendar.shape[0]} rows and {seattle_calendar.shape[1]} columns')
print(f'The Seattle listings dataset has {seattle_listings.shape[0]} rows and {seattle_listings.shape[1]} columns')
print(f'The Seattle reviews dataset has {seattle_reviews.shape[0]} rows and {seattle_reviews.shape[1]} columns')

The Seattle calendar dataset has 1393570 rows and 4 columns
The Seattle listings dataset has 3818 rows and 92 columns
The Seattle reviews dataset has 84849 rows and 6 columns


In [4]:
seattle_calendar.head(15)

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN
5,241032,2016-01-09,f,NaN
6,241032,2016-01-10,f,NaN
7,241032,2016-01-11,f,NaN
8,241032,2016-01-12,f,NaN
9,241032,2016-01-13,t,$85.00


In [5]:
seattle_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1393570 non-null  int64 
 1   date        1393570 non-null  object
 2   available   1393570 non-null  object
 3   price       934542 non-null   object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


In [6]:
seattle_listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [7]:
seattle_listings.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,...,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,...,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,...,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,...,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,...,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,...,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,...,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


In [8]:
seattle_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

In [9]:
seattle_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [10]:
seattle_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   listing_id     84849 non-null  int64 
 1   id             84849 non-null  int64 
 2   date           84849 non-null  object
 3   reviewer_id    84849 non-null  int64 
 4   reviewer_name  84849 non-null  object
 5   comments       84831 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


# Data cleaning and encoding


### Step 1: columns analysis

The number of columns in the listings dataset is reduced to those that are actually useful for answering the three questions.

In [11]:
def df_cleaning(df):
    
    '''
    INPUT
        df: The LISTINGS dataframe to clean
    OUTPUT
        df: The LISTINGS dataframe final version
    '''
    
    # 1 - Reduce the columns of the dataframe
    
    columns = ['id',                             
                'experiences_offered',                                
                'host_since',                     
                'host_response_rate',             
                'host_is_superhost',              
                'host_has_profile_pic',           
                'host_identity_verified',         
                'neighbourhood',                  
                'latitude',                       
                'longitude',                      
                'property_type',                  
                'room_type',                      
                'accommodates',                   
                'bathrooms',                      
                'bedrooms',                       
                'beds',                                             
                'amenities',                      
                'price',                          
                'security_deposit',               
                'cleaning_fee',                                                                
                'number_of_reviews',              
                'review_scores_rating',           
                'review_scores_accuracy',         
                'review_scores_cleanliness',      
                'review_scores_checkin',          
                'review_scores_communication',    
                'review_scores_location',         
                'review_scores_value',            
                'instant_bookable',               
                'cancellation_policy',            
                'require_guest_profile_picture',  
                'require_guest_phone_verification'
    ]
    
    df = df[columns]
    
    # 2 - Change some columns type
    
    # Dates
    df.loc[:,'host_since'] = pd.to_datetime(df['host_since'])
    
    # String to float
    str_to_flt = ['host_response_rate',
                  'price', 
                  'security_deposit', 
                  'cleaning_fee']
    for col in str_to_flt:
        df.loc[:,col] = df[col].map(lambda x: x.strip('%$').replace('.00','').replace(',',''), na_action='ignore').astype('float')       
    
    # String to bool
    str_to_bol = ['host_is_superhost', 
                  'host_has_profile_pic', 
                  'host_identity_verified', 
                  'instant_bookable',                            
                  'require_guest_profile_picture',   
                  'require_guest_phone_verification' ]
    tf = {'t': True, 'f': False}
    
    for col in str_to_bol:
        df.loc[:,col] = df[col].map(tf).astype('bool')
        
#     Amenities: I will create a new column, Amenities_num, counting the number of amenities offered by the host.
#     An alternative would be to split the list of amenities and create a dummy variable for each of them.
#     This solution would crate a large number of new columns, about 15 to 20, and it won't help to answer the questions.
#     This kind of analysis is suitable for amenities-tergeted investigations.
    
    # A trick to count the amenities in the string is to count the commas that separate each entry, plus one.
    
    df['n_amenities'] = df['amenities'].map(lambda x: x.strip('{}').count(',')+1)
    df = df.drop('amenities', axis=1)
        
    return df

In [12]:
seattle_listings_reduced = df_cleaning(seattle_listings)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-11-ac681b9250a3>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_amenities'] = df['amenities'].map(lambda x: x.strip('{}').count(',')+1)


In [13]:
seattle_listings_reduced.head()

,id,experiences_offered,host_since,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,n_amenities
0,241032,none,2011-08-11,96.0,False,True,True,Queen Anne,47.636289,-122.371025,...,10.0,10.0,10.0,9.0,10.0,False,moderate,False,False,10
1,953595,none,2013-02-21,98.0,True,True,True,Queen Anne,47.639123,-122.365666,...,10.0,10.0,10.0,10.0,10.0,False,strict,True,True,16
2,3308979,none,2014-06-12,67.0,False,True,True,Queen Anne,47.629724,-122.369483,...,10.0,10.0,10.0,10.0,10.0,False,strict,False,False,21
3,7421966,none,2013-11-06,NaN,False,True,True,Queen Anne,47.638473,-122.369279,...,NaN,NaN,NaN,NaN,NaN,False,flexible,False,False,14
4,278830,none,2011-11-29,100.0,False,True,True,Queen Anne,47.632918,-122.372471,...,9.0,10.0,10.0,9.0,9.0,False,strict,False,False,13


In [14]:
# The column 'experiences_offered' is populate of 'None' only, so it will be dropped

print(seattle_listings_reduced['experiences_offered'].value_counts())
seattle_listings_reduced = seattle_listings_reduced.drop('experiences_offered', axis=1)

none    3818
Name: experiences_offered, dtype: int64


### Step 2: Deal with missing values  

Depending on the number of missing values in each column, and on what kind of data each column stores, I will adopt a different strategy for dealing with missing data.

In [15]:
seattle_listings_reduced.isnull().sum()

id                                     0
host_since                             2
host_response_rate                   523
host_is_superhost                      0
host_has_profile_pic                   0
host_identity_verified                 0
neighbourhood                        416
latitude                               0
longitude                              0
property_type                          1
room_type                              0
accommodates                           0
bathrooms                             16
bedrooms                               6
beds                                   1
price                                  0
security_deposit                    1952
cleaning_fee                        1030
number_of_reviews                      0
review_scores_rating                 647
review_scores_accuracy               658
review_scores_cleanliness            653
review_scores_checkin                658
review_scores_communication          651
review_scores_lo

In [16]:
# Three columns has few missing data, so I can drop these rows

row_drop = ['host_since', 'property_type', 'bathrooms', 'bedrooms', 'beds']

seattle_listings_reduced = seattle_listings_reduced.dropna(subset=row_drop, how='any')

In [17]:
# The neighbourhood column has 416 missing values that can't be filled with some other value.
# host_response_rate, and the review scores have about 530 missing data, that is about 14% whole data.
# I prefer to drop these rows rather than filling them (e.g. with the review score average)
# because the dataset would then have 14% of rows having the same review score.

miss_to_drop = ['neighbourhood',
                'host_response_rate',
                'review_scores_rating',
                'review_scores_accuracy',
                'review_scores_cleanliness',
                'review_scores_checkin', 
                'review_scores_communication', 
                'review_scores_location', 
                'review_scores_value']
seattle_listings_reduced = seattle_listings_reduced.dropna(subset=miss_to_drop, how='any')

In [18]:
# Security deposit and cleaning fee have 1682 and 879 missing values. 
# I will consider them as if the fee is $0

seattle_listings_reduced['security_deposit'] = seattle_listings_reduced['security_deposit'].fillna(0)
seattle_listings_reduced['cleaning_fee'] = seattle_listings_reduced['cleaning_fee'].fillna(0)

In [19]:
seattle_listings_reduced.isnull().sum()

id                                  0
host_since                          0
host_response_rate                  0
host_is_superhost                   0
host_has_profile_pic                0
host_identity_verified              0
neighbourhood                       0
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                           0
bedrooms                            0
beds                                0
price                               0
security_deposit                    0
cleaning_fee                        0
number_of_reviews                   0
review_scores_rating                0
review_scores_accuracy              0
review_scores_cleanliness           0
review_scores_checkin               0
review_scores_communication         0
review_scores_location              0
review_scores_value                 0
instant_book

In [20]:
seattle_listings_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2555 entries, 0 to 3814
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   id                                2555 non-null   int64         
 1   host_since                        2555 non-null   datetime64[ns]
 2   host_response_rate                2555 non-null   float64       
 3   host_is_superhost                 2555 non-null   bool          
 4   host_has_profile_pic              2555 non-null   bool          
 5   host_identity_verified            2555 non-null   bool          
 6   neighbourhood                     2555 non-null   object        
 7   latitude                          2555 non-null   float64       
 8   longitude                         2555 non-null   float64       
 9   property_type                     2555 non-null   object        
 10  room_type                         2555 non-null 

### Step 3: Dealing with categorical variables  

The categorical columns are first cleaned from low counts, and then encoded.

In [21]:
categorical = seattle_listings_reduced.select_dtypes(include=['object'])
categorical.value_counts()

neighbourhood              property_type  room_type        cancellation_policy
Belltown                   Apartment      Entire home/apt  strict                 116
Capitol Hill               Apartment      Entire home/apt  strict                  55
                                                           moderate                50
Central Business District  Apartment      Entire home/apt  strict                  45
First Hill                 Apartment      Entire home/apt  strict                  37
                                                                                 ... 
Laurelhurst                House          Entire home/apt  moderate                 1
                           Cabin          Entire home/apt  moderate                 1
                           Apartment      Private room     moderate                 1
                                          Entire home/apt  strict                   1
Yesler Terrace             House          Entire home/apt  st

In [22]:
# neighbourhood has a very large variety of values. Some neighbourhoods, though, have very few listings. 
# I will delate the row having less than 10 listings in a neighbourhood
# (https://stackoverflow.com/questions/49735683/python-removing-rows-on-count-condition).

# After the rows with low counts are removed, the same columns can be encoded using dummy variables.

row_to_encode = ['neighbourhood', 'property_type', 'room_type', 'cancellation_policy']

for col in row_to_encode:
    n_counts = categorical[col].value_counts()
    seattle_listings_reduced = seattle_listings_reduced[~seattle_listings_reduced[col].isin(n_counts[n_counts < 10].index)]

seattle_listings_2 = pd.get_dummies(seattle_listings_reduced,columns=row_to_encode)

In [23]:
seattle_listings_2.head()

,id,host_since,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,...,property_type_House,property_type_Loft,property_type_Other,property_type_Townhouse,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict
0,241032,2011-08-11,96.0,False,True,True,47.636289,-122.371025,4,1.0,...,0,0,0,0,1,0,0,0,1,0
1,953595,2013-02-21,98.0,True,True,True,47.639123,-122.365666,4,1.0,...,0,0,0,0,1,0,0,0,0,1
2,3308979,2014-06-12,67.0,False,True,True,47.629724,-122.369483,11,4.5,...,1,0,0,0,1,0,0,0,0,1
4,278830,2011-11-29,100.0,False,True,True,47.632918,-122.372471,6,2.0,...,1,0,0,0,1,0,0,0,0,1
6,1909058,2012-05-30,100.0,True,True,True,47.636605,-122.368519,2,1.0,...,1,0,0,0,0,1,0,0,1,0


In [24]:
seattle_listings_2.describe()

,id,host_response_rate,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,...,property_type_House,property_type_Loft,property_type_Other,property_type_Townhouse,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict
count,2.382000e+03,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,...,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000
mean,4.868518e+06,95.310243,47.629121,-122.332892,3.452561,1.262594,1.303946,1.776238,126.600336,145.160789,...,0.453820,0.009236,0.004198,0.025609,0.682200,0.286314,0.031486,0.227540,0.350966,0.421495
std,2.823953e+06,10.860378,0.039620,0.030926,2.025887,0.590417,0.869943,1.173653,89.132262,221.082448,...,0.497967,0.095679,0.064671,0.157998,0.465719,0.452133,0.174664,0.419332,0.477372,0.493902
min,4.291000e+03,17.000000,47.508394,-122.417219,1.000000,0.000000,0.000000,1.000000,25.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.418963e+06,98.000000,47.609782,-122.352787,2.000000,1.000000,1.000000,1.000000,75.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.949216e+06,100.000000,47.623059,-122.329261,3.000000,1.000000,1.000000,1.000000,100.000000,100.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.386046e+06,100.000000,47.661251,-122.311136,4.000000,1.000000,2.000000,2.000000,150.000000,250.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,1.011834e+07,100.000000,47.733316,-122.257085,16.000000,8.000000,6.000000,15.000000,999.000000,2500.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
seattle_listings_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2382 entries, 0 to 3814
Data columns (total 91 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       2382 non-null   int64         
 1   host_since                               2382 non-null   datetime64[ns]
 2   host_response_rate                       2382 non-null   float64       
 3   host_is_superhost                        2382 non-null   bool          
 4   host_has_profile_pic                     2382 non-null   bool          
 5   host_identity_verified                   2382 non-null   bool          
 6   latitude                                 2382 non-null   float64       
 7   longitude                                2382 non-null   float64       
 8   accommodates                             2382 non-null   int64         
 9   bathrooms                                

### Step 4: The other two datasets.

**seattle_reviews:** The information in this dataset is not relevant for this analysis, so it won't be used.  

**seattle_calendar:** This dataset can help answering the first question: considering the reviews for each property, how busy will it be until the end of year?
I will count the number of dates each listing is already booked, and I will add it as a new column to the seattle_listings data frame.

In [26]:
seattle_calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [27]:
# The 'available' column is conveted to 1 (the room is not available - it was booked) and 0 (the room is still unbooked)

tf = {'t': 0, 'f': 1}
seattle_calendar.loc[:,'available'] = seattle_calendar['available'].map(tf)

In [28]:
seattle_calendar = seattle_calendar.rename(columns={'available':'number_of_bookings'})

In [29]:
seattle_calendar.head()

,listing_id,date,number_of_bookings,price
0,241032,2016-01-04,0,$85.00
1,241032,2016-01-05,0,$85.00
2,241032,2016-01-06,1,NaN
3,241032,2016-01-07,1,NaN
4,241032,2016-01-08,1,NaN


In [30]:
summed_df = seattle_calendar.groupby('listing_id').sum().reset_index()
summed_df = summed_df.rename(columns={'listing_id':'id'})
seattle_final = pd.merge(seattle_listings_2, summed_df, on ='id')
seattle_final = seattle_final.drop('id', axis=1)

In [31]:
seattle_final.head()

,host_since,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,...,property_type_Loft,property_type_Other,property_type_Townhouse,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,number_of_bookings
0,2011-08-11,96.0,False,True,True,47.636289,-122.371025,4,1.0,1.0,...,0,0,0,1,0,0,0,1,0,19
1,2013-02-21,98.0,True,True,True,47.639123,-122.365666,4,1.0,1.0,...,0,0,0,1,0,0,0,0,1,74
2,2014-06-12,67.0,False,True,True,47.629724,-122.369483,11,4.5,5.0,...,0,0,0,1,0,0,0,0,1,145
3,2011-11-29,100.0,False,True,True,47.632918,-122.372471,6,2.0,3.0,...,0,0,0,1,0,0,0,0,1,0
4,2012-05-30,100.0,True,True,True,47.636605,-122.368519,2,1.0,1.0,...,0,0,0,0,1,0,0,1,0,19


In [32]:
seattle_listings_reduced = pd.merge(seattle_listings_reduced, summed_df, on ='id')
seattle_listings_reduced = seattle_listings_reduced.drop('id', axis=1)

In [33]:
seattle_listings_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2382 entries, 0 to 2381
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   host_since                        2382 non-null   datetime64[ns]
 1   host_response_rate                2382 non-null   float64       
 2   host_is_superhost                 2382 non-null   bool          
 3   host_has_profile_pic              2382 non-null   bool          
 4   host_identity_verified            2382 non-null   bool          
 5   neighbourhood                     2382 non-null   object        
 6   latitude                          2382 non-null   float64       
 7   longitude                         2382 non-null   float64       
 8   property_type                     2382 non-null   object        
 9   room_type                         2382 non-null   object        
 10  accommodates                      2382 non-null 

In [34]:
seattle_listings_reduced['avg_reviews'] = ((seattle_listings_reduced['review_scores_accuracy'] +
                                           seattle_listings_reduced['review_scores_cleanliness'] +
                                           seattle_listings_reduced['review_scores_checkin'] +
                                           seattle_listings_reduced['review_scores_communication'] +
                                           seattle_listings_reduced['review_scores_location'] +
                                           seattle_listings_reduced['review_scores_value']) / 6).round(2)

In [35]:
seattle_final['avg_reviews'] = ((seattle_final['review_scores_accuracy'] +
                                           seattle_final['review_scores_cleanliness'] +
                                           seattle_final['review_scores_checkin'] +
                                           seattle_final['review_scores_communication'] +
                                           seattle_final['review_scores_location'] +
                                           seattle_final['review_scores_value']) / 6).round(2)

In [36]:
seattle_listings_reduced.head()

,host_since,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,property_type,room_type,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,n_amenities,number_of_bookings,avg_reviews
0,2011-08-11,96.0,False,True,True,Queen Anne,47.636289,-122.371025,Apartment,Entire home/apt,...,10.0,9.0,10.0,False,moderate,False,False,10,19,9.83
1,2013-02-21,98.0,True,True,True,Queen Anne,47.639123,-122.365666,Apartment,Entire home/apt,...,10.0,10.0,10.0,False,strict,True,True,16,74,10.00
2,2014-06-12,67.0,False,True,True,Queen Anne,47.629724,-122.369483,House,Entire home/apt,...,10.0,10.0,10.0,False,strict,False,False,21,145,10.00
3,2011-11-29,100.0,False,True,True,Queen Anne,47.632918,-122.372471,House,Entire home/apt,...,10.0,9.0,9.0,False,strict,False,False,13,0,9.33
4,2012-05-30,100.0,True,True,True,Queen Anne,47.636605,-122.368519,House,Private room,...,10.0,10.0,10.0,False,moderate,False,False,7,19,10.00


In [37]:
seattle_final.head()

,host_since,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,...,property_type_Other,property_type_Townhouse,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,number_of_bookings,avg_reviews
0,2011-08-11,96.0,False,True,True,47.636289,-122.371025,4,1.0,1.0,...,0,0,1,0,0,0,1,0,19,9.83
1,2013-02-21,98.0,True,True,True,47.639123,-122.365666,4,1.0,1.0,...,0,0,1,0,0,0,0,1,74,10.00
2,2014-06-12,67.0,False,True,True,47.629724,-122.369483,11,4.5,5.0,...,0,0,1,0,0,0,0,1,145,10.00
3,2011-11-29,100.0,False,True,True,47.632918,-122.372471,6,2.0,3.0,...,0,0,1,0,0,0,0,1,0,9.33
4,2012-05-30,100.0,True,True,True,47.636605,-122.368519,2,1.0,1.0,...,0,0,0,1,0,0,1,0,19,10.00


In [38]:
seattle_final.head()

,host_since,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,...,property_type_Other,property_type_Townhouse,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,number_of_bookings,avg_reviews
0,2011-08-11,96.0,False,True,True,47.636289,-122.371025,4,1.0,1.0,...,0,0,1,0,0,0,1,0,19,9.83
1,2013-02-21,98.0,True,True,True,47.639123,-122.365666,4,1.0,1.0,...,0,0,1,0,0,0,0,1,74,10.00
2,2014-06-12,67.0,False,True,True,47.629724,-122.369483,11,4.5,5.0,...,0,0,1,0,0,0,0,1,145,10.00
3,2011-11-29,100.0,False,True,True,47.632918,-122.372471,6,2.0,3.0,...,0,0,1,0,0,0,0,1,0,9.33
4,2012-05-30,100.0,True,True,True,47.636605,-122.368519,2,1.0,1.0,...,0,0,0,1,0,0,1,0,19,10.00


In [39]:
seattle_listings_reduced.head()

,host_since,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,property_type,room_type,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,n_amenities,number_of_bookings,avg_reviews
0,2011-08-11,96.0,False,True,True,Queen Anne,47.636289,-122.371025,Apartment,Entire home/apt,...,10.0,9.0,10.0,False,moderate,False,False,10,19,9.83
1,2013-02-21,98.0,True,True,True,Queen Anne,47.639123,-122.365666,Apartment,Entire home/apt,...,10.0,10.0,10.0,False,strict,True,True,16,74,10.00
2,2014-06-12,67.0,False,True,True,Queen Anne,47.629724,-122.369483,House,Entire home/apt,...,10.0,10.0,10.0,False,strict,False,False,21,145,10.00
3,2011-11-29,100.0,False,True,True,Queen Anne,47.632918,-122.372471,House,Entire home/apt,...,10.0,9.0,9.0,False,strict,False,False,13,0,9.33
4,2012-05-30,100.0,True,True,True,Queen Anne,47.636605,-122.368519,House,Private room,...,10.0,10.0,10.0,False,moderate,False,False,7,19,10.00


In [40]:
seattle_listings_reduced.to_csv('Seattle/seattle_listings_preprocessed.csv')

In [41]:
seattle_final.to_csv('Seattle/seattle_listings_preprocessed_encoded.csv')